In [1]:
import sagemaker
import boto3
import sys
import os
import glob
import re
import subprocess
from IPython.display import HTML
import time
from time import gmtime, strftime
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework

In [2]:
# Training settings
s3_name = "sagemaker-cmcollander"
session_name = "reacher-hyper-final"
instance_type = "ml.m4.xlarge"
max_jobs = 565
max_parallel_jobs = 4

# Required variables and objects
sage_session = sagemaker.session.Session()
s3_output_path = f's3://{s3_name}/'

# IAM role
try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()
print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::628874203695:role/service-role/AmazonSageMaker-ExecutionRole-20200423T161797


In [3]:
%%time

cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = f"sagemaker-roboschool-ray-{cpu_or_gpu}"
docker_build_args = {
    'CPU_OR_GPU': cpu_or_gpu, 
    'AWS_REGION': boto3.Session().region_name,
}
custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
print(f"Using ECR image {custom_image_name}")

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Logged into ECR
Building docker image sagemaker-roboschool-ray-cpu from Dockerfile
$ docker build -t sagemaker-roboschool-ray-cpu -f Dockerfile . --build-arg CPU_OR_GPU=cpu --build-arg AWS_REGION=us-east-2
Sending build context to Docker daemon  709.1kB
Step 1/14 : ARG CPU_OR_GPU
Step 2/14 : ARG AWS_REGION
Step 3/14 : FROM 520713654638.dkr.ecr.${AWS_REGION}.amazonaws.com/sagemaker-rl-tensorflow:ray0.6.5-${CPU_OR_GPU}-py3
ray0.6.5-cpu-py3: Pulling from sagemaker-rl-tensorflow
7e6591854262: Pulling fs layer
089d60cb4e0a: Pulling fs layer
9c461696bc09: Pulling fs layer
45085432511a: Pulling fs layer
133ef7c773de: Pulling fs layer
536fa53fdaf4: Pulling fs layer
9b060350118c: Pulli

1 upgraded, 126 newly installed, 0 to remove and 94 not upgraded.
Need to get 31.2 MB of archives.
After this operation, 162 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/main amd64 libpopt0 amd64 1.16-10 [26.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 cmake-data all 3.5.1-1ubuntu3 [1121 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial/main amd64 liblzo2-2 amd64 2.08-1.2 [48.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 zlib1g amd64 1:1.2.8.dfsg-2ubuntu4.3 [51.2 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libarchive13 amd64 3.1.2-11ubuntu0.16.04.8 [262 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libcurl3 amd64 7.47.0-1ubuntu2.14 [186 kB]
Get:7 http://archive.ubuntu.com/ubuntu xenial/main amd64 libjsoncpp1 amd64 1.7.2-1 [73.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 cmake amd64 3.5.1-1ubuntu3 [2623 kB]
Get:9 http://a

Get:81 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-render0-dev amd64 1.11.1-1ubuntu1 [15.3 kB]
Get:82 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-randr0-dev amd64 1.11.1-1ubuntu1 [18.2 kB]
Get:83 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-shape0-dev amd64 1.11.1-1ubuntu1 [6900 B]
Get:84 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-xfixes0-dev amd64 1.11.1-1ubuntu1 [11.2 kB]
Get:85 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-sync-dev amd64 1.11.1-1ubuntu1 [10.1 kB]
Get:86 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb-present-dev amd64 1.11.1-1ubuntu1 [6618 B]
Get:87 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxshmfence-dev amd64 1.2-1 [3676 B]
Get:88 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-xcb-dev amd64 2:1.6.3-1ubuntu2.1 [9718 B]
Get:89 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libwayland-cursor0 amd64 1.12.0-1~ubuntu16.04.3 [10.1 kB]
Get:90 http://arch

Selecting previously unselected package libboost-filesystem1.58.0:amd64.
Preparing to unpack .../libboost-filesystem1.58.0_1.58.0+dfsg-5ubuntu3.1_amd64.deb ...
Unpacking libboost-filesystem1.58.0:amd64 (1.58.0+dfsg-5ubuntu3.1) ...
Selecting previously unselected package libmircore1:amd64.
Preparing to unpack .../libmircore1_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpacking libmircore1:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Selecting previously unselected package libmircommon7:amd64.
Preparing to unpack .../libmircommon7_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpacking libmircommon7:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Selecting previously unselected package libprotobuf-lite9v5:amd64.
Preparing to unpack .../libprotobuf-lite9v5_2.6.1-1.3_amd64.deb ...
Unpacking libprotobuf-lite9v5:amd64 (2.6.1-1.3) ...
Selecting previously unselected package libmirprotobuf3:amd64.
Preparing to unpack .../libmirprotobuf3_0.26.3+16.04.20170605-0ubuntu1.1_amd64.deb ...
Unpackin

Selecting previously unselected package x11proto-kb-dev.
Preparing to unpack .../x11proto-kb-dev_1.0.7-0ubuntu1_all.deb ...
Unpacking x11proto-kb-dev (1.0.7-0ubuntu1) ...
Selecting previously unselected package xtrans-dev.
Preparing to unpack .../xtrans-dev_1.3.5-1_all.deb ...
Unpacking xtrans-dev (1.3.5-1) ...
Selecting previously unselected package libpthread-stubs0-dev:amd64.
Preparing to unpack .../libpthread-stubs0-dev_0.3-4_amd64.deb ...
Unpacking libpthread-stubs0-dev:amd64 (0.3-4) ...
Selecting previously unselected package libxcb1-dev:amd64.
Preparing to unpack .../libxcb1-dev_1.11.1-1ubuntu1_amd64.deb ...
Unpacking libxcb1-dev:amd64 (1.11.1-1ubuntu1) ...
Selecting previously unselected package libx11-dev:amd64.
Preparing to unpack .../libx11-dev_2%3a1.6.3-1ubuntu2.1_amd64.deb ...
Unpacking libx11-dev:amd64 (2:1.6.3-1ubuntu2.1) ...
Selecting previously unselected package x11proto-xext-dev.
Preparing to unpack .../x11proto-xext-dev_7.3.0-1_all.deb ...
Unpacking x11proto-xext-de

Preparing to unpack .../libqt5opengl5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5opengl5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5printsupport5:amd64.
Preparing to unpack .../libqt5printsupport5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5printsupport5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5sql5:amd64.
Preparing to unpack .../libqt5sql5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5sql5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5test5:amd64.
Preparing to unpack .../libqt5test5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5test5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package libqt5xml5:amd64.
Preparing to unpack .../libqt5xml5_5.5.1+dfsg-16ubuntu7.7_amd64.deb ...
Unpacking libqt5xml5:amd64 (5.5.1+dfsg-16ubuntu7.7) ...
Selecting previously unselected package qtchooser.
Preparing to unpack .../qtchooser_52-gae5

Setting up libxshmfence-dev:amd64 (1.2-1) ...
Setting up libx11-xcb-dev:amd64 (2:1.6.3-1ubuntu2.1) ...
Setting up libwayland-cursor0:amd64 (1.12.0-1~ubuntu16.04.3) ...
Setting up libwayland-bin (1.12.0-1~ubuntu16.04.3) ...
Setting up libwayland-dev:amd64 (1.12.0-1~ubuntu16.04.3) ...
Setting up libmircore-dev:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up zlib1g-dev:amd64 (1:1.2.8.dfsg-2ubuntu4.3) ...
Setting up libprotobuf9v5:amd64 (2.6.1-1.3) ...
Setting up libprotobuf-dev:amd64 (2.6.1-1.3) ...
Setting up libxkbcommon-dev (0.5.0-1ubuntu2.1) ...
Setting up libmircommon-dev:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up libmircookie2:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up libmircookie-dev:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up libmirclient-dev:amd64 (0.26.3+16.04.20170605-0ubuntu1.1) ...
Setting up libegl1-mesa-dev:amd64 (18.0.5-0ubuntu0~16.04.1) ...
Setting up libgles2-mesa:amd64 (18.0.5-0ubuntu0~16.04.1) ...
Setting up libgles2-mesa-

Preparing to unpack .../python2.7-minimal_2.7.12-1ubuntu0~16.04.11_amd64.deb ...
Unpacking python2.7-minimal (2.7.12-1ubuntu0~16.04.11) ...
Selecting previously unselected package python-minimal.
Preparing to unpack .../python-minimal_2.7.12-1~16.04_amd64.deb ...
Unpacking python-minimal (2.7.12-1~16.04) ...
Selecting previously unselected package libpython2.7-stdlib:amd64.
Preparing to unpack .../libpython2.7-stdlib_2.7.12-1ubuntu0~16.04.11_amd64.deb ...
Unpacking libpython2.7-stdlib:amd64 (2.7.12-1ubuntu0~16.04.11) ...
Selecting previously unselected package python2.7.
Preparing to unpack .../python2.7_2.7.12-1ubuntu0~16.04.11_amd64.deb ...
Unpacking python2.7 (2.7.12-1ubuntu0~16.04.11) ...
Selecting previously unselected package libpython-stdlib:amd64.
Preparing to unpack .../libpython-stdlib_2.7.12-1~16.04_amd64.deb ...
Unpacking libpython-stdlib:amd64 (2.7.12-1~16.04) ...
Processing triggers for mime-support (3.59ubuntu1) ...
Setting up libpython2.7-minimal:amd64 (2.7.12-1ubuntu0~

      Successfully uninstalled pip-18.1
Removing intermediate container 24fd1384a197
 ---> 73be2c755d7a
Step 9/14 : RUN pip install --upgrade     pip     setuptools     setproctitle     lz4     psutil
 ---> Running in b0d538358cfe
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1)
  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp36-cp36m-linux_x86_64.whl size=26439 sha256=2a8dee90de65728f99c228d1b5e1cbd5f0946c5db1ca0282ca3841a7209d51d5
  Stored in directory: /root/.cache/pip/wheels/21/08/bf/dfab9a2a12aa6c3f1f1785dd5af62d734ab48f1b4b1f7d99d3
  Created wheel for psutil: filename=psutil-5.7.0-cp36-cp36m-linux_x86_64.whl size=259953 sha256=7fdc09983fc7676f40722967a0c35c8a65dc0b76f86c5295d87a8eecf2237625
  Stored in directory: /root/.cache/pip/wheels/a1/d9/f2/b5620c01e9b3e858c6877b1045fda5b115cf7df6490f883382
Successfully built setproctitle psutil
ERROR: sagemaker-containers 2.4.6.post0 has requirement psutil==5.4.8, but you'll have psutil 5

  Created wheel for inotify-simple: filename=inotify_simple-1.2.1-py3-none-any.whl size=8203 sha256=52d7a12395d40087fa178212d6ed320a875aba4faef155e7147581c9caf7faa9
  Stored in directory: /root/.cache/pip/wheels/a5/34/f6/5d4eae9a326dc637344cd68353ddd119e9f6f575e925502888
Successfully built sagemaker-containers retrying inotify-simple
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 0.15.2
    Uninstalling Werkzeug-0.15.2:
      Successfully uninstalled Werkzeug-0.15.2
  Attempting uninstall: flask
    Found existing installation: Flask 1.0.2
    Uninstalling Flask-1.0.2:
      Successfully uninstalled Flask-1.0.2
  Attempting uninstall: inotify-simple
    Found existing installation: inotify-simple 1.1.8
    Uninstalling inotify-simple-1.1.8:
      Successfully uninstalled inotify-simple-1.1.8
  Attempting uninstall: scipy
    Found existing installation: scipy 1.2.1
    Uninstalling scipy-1.2.1:
      Successfully uninstalled scipy-1.2.1
  Attempting uninstal

In [4]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# The hyperparameters we're going to tune
hyperparameter_ranges = {
    'rl.training.config.lambda': CategoricalParameter([0.9,0.95,1.0]),
    'rl.training.config.sgd_minibatch_size': CategoricalParameter([128,512]),
    'rl.training.config.num_sgd_iter': CategoricalParameter([5,10]),
    'rl.training.config.clip_param': CategoricalParameter([0.1,0.3,0.5]),
    'rl.training.config.vf_clip_param': CategoricalParameter([5,10]),
    'rl.training.config.kl_target': CategoricalParameter([0.01,0.05])
}

# The hyperparameters that are the same for all jobs
static_hyperparameters = {
    "rl.training.stop.training_iteration": 250,  # Tell each training job to stop after 250 training iterations
    "model_dir": "model",
}

In [5]:
metric_definitions = RLEstimator.default_metric_definitions(RLToolkit.RAY)
estimator = RLEstimator(entry_point="train-reacher.py",
                        source_dir='src',
                        dependencies=["common/sagemaker_rl"],
                        image_name=custom_image_name,
                        role=role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=session_name,
                        metric_definitions=metric_definitions,
                        hyperparameters=static_hyperparameters,
                    )

tuner = HyperparameterTuner(estimator,
                            objective_metric_name='episode_reward_mean',
                            objective_type='Maximize',
                            hyperparameter_ranges=hyperparameter_ranges,
                            metric_definitions=metric_definitions,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs,
                            base_tuning_job_name=f"{session_name}-tuning",
                           )
tuner.fit()

2020-04-28 22:53:55 Starting - Starting the training job...
2020-04-28 22:53:56 Starting - Launching requested ML instances......
2020-04-28 22:54:56 Starting - Preparing the instances for training...
2020-04-28 22:55:48 Downloading - Downloading input data
2020-04-28 22:55:48 Training - Downloading the training image.........
2020-04-28 22:57:19 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-28 22:57:22,692 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-04-28 22:57:22,698 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-28 22:57:22,847 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-28 22:57:22,864 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-28 22:57:22,882 sagemaker-containers INFO     N

#033#033(pid=88)#033 2020-04-28 22:57:54,644#011INFO policy_evaluator.py:278 -- Creating policy evaluation worker 3 on CPU (please ignore any CUDA init errors)
#033#033(pid=85)#033 2020-04-28 22:57:55,019#011INFO policy_evaluator.py:278 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
#033#033(pid=88)#033 /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
#033#033(pid=88)#033   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
#033#033(pid=85)#033 /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
#033#033(pid=85)#033   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
#033#033(pid=87)#033 2020-04-28 22

Result for PPO_RoboschoolReacher-v1_0:
  custom_metrics: {}
  date: 2020-04-28_23-02-12
  done: false
  episode_len_mean: 150.0
  episode_reward_max: 14.905034425201467
  episode_reward_mean: -11.31371284978314
  episode_reward_min: -45.39006065165898
  episodes_this_iter: 168
  episodes_total: 840
  experiment_id: fe8512598411417ba3ca9ed05f5ca399
  hostname: ip-10-0-76-163.us-east-2.compute.internal
  info:
    default:
      cur_kl_coeff: 0.0625
      cur_lr: 9.999999747378752e-05
      entropy: 2.8059897422790527
      kl: 0.00694463262334466
      policy_loss: -0.0038903439417481422
      total_loss: 52.98564147949219
      vf_explained_var: 0.2760685682296753
      vf_loss: 52.989097595214844
    grad_time_ms: 14195.294
    load_time_ms: 12.083
    num_steps_sampled: 126000
    num_steps_trained: 125000
    sample_time_ms: 38355.499
    update_time_ms: 127.803
  iterations_since_restore: 5
  node_ip: 10.0.76.163
  num_healthy_workers: 3
  num_metric_batches_dropped: 0
  off_policy

Result for PPO_RoboschoolReacher-v1_0:
  custom_metrics: {}
  date: 2020-04-28_23-05-12
  done: false
  episode_len_mean: 150.0
  episode_reward_max: 22.9076317771392
  episode_reward_mean: -7.492871290711091
  episode_reward_min: -35.61563811934083
  episodes_this_iter: 168
  episodes_total: 1512
  experiment_id: fe8512598411417ba3ca9ed05f5ca399
  hostname: ip-10-0-76-163.us-east-2.compute.internal
  info:
    default:
      cur_kl_coeff: 0.03125
      cur_lr: 9.999999747378752e-05
      entropy: 2.7729787826538086
      kl: 0.0068600536324083805
      policy_loss: -0.004566876217722893
      total_loss: 24.991500854492188
      vf_explained_var: 0.3719639182090759
      vf_loss: 24.9958553314209
    grad_time_ms: 14134.353
    load_time_ms: 7.403
    num_steps_sampled: 226800
    num_steps_trained: 225000
    sample_time_ms: 35031.423
    update_time_ms: 73.552
  iterations_since_restore: 9
  node_ip: 10.0.76.163
  num_healthy_workers: 3
  num_metric_batches_dropped: 0
  off_policy_e

Result for PPO_RoboschoolReacher-v1_0:
  custom_metrics: {}
  date: 2020-04-28_23-08-14
  done: false
  episode_len_mean: 150.0
  episode_reward_max: 23.922160760559464
  episode_reward_mean: -4.910603819323595
  episode_reward_min: -30.389007980715615
  episodes_this_iter: 168
  episodes_total: 2184
  experiment_id: fe8512598411417ba3ca9ed05f5ca399
  hostname: ip-10-0-76-163.us-east-2.compute.internal
  info:
    default:
      cur_kl_coeff: 0.03125
      cur_lr: 9.999999747378752e-05
      entropy: 2.7368133068084717
      kl: 0.007559581194072962
      policy_loss: -0.006077935919165611
      total_loss: 17.761337280273438
      vf_explained_var: 0.5576774477958679
      vf_loss: 17.767179489135742
    grad_time_ms: 13995.997
    load_time_ms: 1.568
    num_steps_sampled: 327600
    num_steps_trained: 325000
    sample_time_ms: 31336.018
    update_time_ms: 7.135
  iterations_since_restore: 13
  node_ip: 10.0.76.163
  num_healthy_workers: 3
  num_metric_batches_dropped: 0
  off_poli

Result for PPO_RoboschoolReacher-v1_0:
  custom_metrics: {}
  date: 2020-04-28_23-11-10
  done: false
  episode_len_mean: 150.0
  episode_reward_max: 23.284787190906474
  episode_reward_mean: -0.7677736423831716
  episode_reward_min: -34.268133864693205
  episodes_this_iter: 168
  episodes_total: 2856
  experiment_id: fe8512598411417ba3ca9ed05f5ca399
  hostname: ip-10-0-76-163.us-east-2.compute.internal
  info:
    default:
      cur_kl_coeff: 0.03125
      cur_lr: 9.999999747378752e-05
      entropy: 2.6738085746765137
      kl: 0.008951305411756039
      policy_loss: -0.008391919545829296
      total_loss: 12.142171859741211
      vf_explained_var: 0.7079110145568848
      vf_loss: 12.150284767150879
    grad_time_ms: 13985.281
    load_time_ms: 1.629
    num_steps_sampled: 428400
    num_steps_trained: 425000
    sample_time_ms: 30530.19
    update_time_ms: 8.118
  iterations_since_restore: 17
  node_ip: 10.0.76.163
  num_healthy_workers: 3
  num_metric_batches_dropped: 0
  off_poli

Saved the checkpoint file /opt/ml/output/intermediate/training/PPO_RoboschoolReacher-v1_0_2020-04-28_22-57-34_rrbwde3/checkpoint_20/checkpoint-20 as /opt/ml/model/checkpoint
Saved the checkpoint file /opt/ml/output/intermediate/training/PPO_RoboschoolReacher-v1_0_2020-04-28_22-57-34_rrbwde3/checkpoint_20/checkpoint-20.tune_metadata as /opt/ml/model/checkpoint.tune_metadata
2020-04-28 23:13:30,684#011WARNING ppo.py:172 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2020-04-28 23:13:30,694#011INFO policy_evaluator.py:278 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2020-04-28 23:13:31,898#011INFO multi_gpu_optim